In [21]:
import snowflake.connector

conn = snowflake.connector.connect(
    user="USERSNOW",
    password="Password123456",
    account="alnwmmv-nj56428",   # 👈 nuevo formato para app.snowflake.com
    warehouse="COMPUTE_WH",
    database="NYCTAXI",
    schema="GOLD",
    role="ACCOUNTADMIN"
)

cur = conn.cursor()
cur.execute("SELECT CURRENT_ACCOUNT(), CURRENT_REGION(), CURRENT_ROLE();")
print(cur.fetchall())



[('LY21080', 'AWS_SA_EAST_1', 'ACCOUNTADMIN')]


Demanda por zona y mes

In [24]:
query_pu = """
SELECT
    TRIP_YEAR,
    TRIP_MONTH,
    PICKUP_ZONE AS ZONE,
    SUM(TOTAL_TRIPS) AS TOTAL_TRIPS
FROM GOLD.FACT_TRIPS
GROUP BY TRIP_YEAR, TRIP_MONTH, PICKUP_ZONE
ORDER BY TRIP_YEAR, TRIP_MONTH, TOTAL_TRIPS DESC
LIMIT 10;
"""

df_pu = pd.read_sql(query_pu, conn)
df_pu.head(10)


C:\Users\GISELLE1\AppData\Local\Temp\ipykernel_30484\3591754541.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_pu = pd.read_sql(query_pu, conn)


,TRIP_YEAR,TRIP_MONTH,ZONE,TOTAL_TRIPS
0,2002,12,Murray Hill,21
1,2024,3,Elmhurst,42
2,2024,3,East Chelsea,21
3,2024,3,Greenwich Village South,21
4,2024,3,Union Sq,21
5,2024,3,JFK Airport,21
6,2024,3,Upper East Side North,21
7,2024,3,East Harlem North,21
8,2024,4,Upper East Side South,3153843
9,2024,4,JFK Airport,3101742


Las zonas con mayor volumen de viajes son Upper East Side South y JFK Airport, ambas con más de 3 millones de viajes en abril de 2024.

Esto indica que Manhattan y el aeropuerto JFK concentran gran parte de la demanda, actuando como polos principales de origen y destino de viajes.

La presencia de zonas con volúmenes bajos (21–42 viajes) sugiere registros antiguos o datos de prueba (como el año 2002).

El patrón general muestra un pico de demanda en primavera (marzo–abril de 2024), consistente con una mayor movilidad urbana postinvierno

Ingresos y propinas

In [27]:
query_2 = """
SELECT
    TRIP_YEAR,
    TRIP_MONTH,
    PICKUP_BOROUGH,
    SUM(TOTAL_REVENUE) AS TOTAL_REVENUE,
    ROUND(AVG(AVG_TIP_PERCENT), 2) AS AVG_TIP_PERCENT
FROM GOLD.FACT_TRIPS
GROUP BY TRIP_YEAR, TRIP_MONTH, PICKUP_BOROUGH
ORDER BY TRIP_YEAR, TRIP_MONTH, TOTAL_REVENUE DESC;
"""

df_2 = pd.read_sql(query_2, conn)
df_2.head(10)


C:\Users\GISELLE1\AppData\Local\Temp\ipykernel_30484\4217145546.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_2 = pd.read_sql(query_2, conn)


,TRIP_YEAR,TRIP_MONTH,PICKUP_BOROUGH,TOTAL_REVENUE,AVG_TIP_PERCENT
0,2002,12,Manhattan,9.601200e+02,23.02
1,2024,3,Queens,3.075030e+03,17.16
2,2024,3,Manhattan,2.139270e+03,11.14
3,2024,4,Manhattan,1.339435e+09,19.16
4,2024,4,Queens,4.539365e+08,9.21
5,2024,4,Brooklyn,2.012927e+07,5.33
6,2024,4,None,6.910651e+06,20.19
7,2024,4,Bronx,5.198681e+06,1.20
8,2024,4,EWR,1.977505e+05,20.10
9,2024,4,Staten Island,4.807824e+04,10.48


Los mayores ingresos se concentran en Manhattan, con más de 1.3 mil millones de dólares en abril de 2024.

Queens ocupa el segundo lugar con 453 millones, seguido de Brooklyn con 20 millones.

En contraste, EWR (Newark Airport) y Staten Island generan ingresos muy bajos.

La columna None refleja registros sin asignación de borough, probablemente por zonas no mapeadas en el lookup.

Las propinas promedio (AVG_TIP_PERCENT) mantienen una correlación con el ingreso total: zonas de mayor valor de viaje → mayor tip %.

Velocidad y congestión

In [29]:
query_3 = """
SELECT
    PICKUP_BOROUGH,
    ROUND(AVG(AVG_DISTANCE_KM / NULLIF(AVG_TRIP_MINUTES / 60, 0) * 0.621371), 2) AS AVG_MPH
FROM GOLD.FACT_TRIPS
GROUP BY PICKUP_BOROUGH
ORDER BY AVG_MPH DESC;
"""

df_3 = pd.read_sql(query_3, conn)
df_3


C:\Users\GISELLE1\AppData\Local\Temp\ipykernel_30484\718176356.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_3 = pd.read_sql(query_3, conn)


,PICKUP_BOROUGH,AVG_MPH
0,None,34.98
1,EWR,22.28
2,Staten Island,16.96
3,Queens,13.42
4,Brooklyn,10.18
5,Manhattan,8.59
6,Bronx,7.32


Las velocidades más altas corresponden a zonas fuera del núcleo urbano, como None (registros sin borough), EWR (22.3 mph) y Staten Island (16.9 mph).

Las más bajas aparecen en Manhattan (8.6 mph) y Bronx (7.3 mph), lo que refleja alta congestión vehicular en zonas céntricas.

Queens y Brooklyn tienen valores intermedios (10–13 mph), típicos de áreas mixtas residenciales–comerciales.

Duración del viaje

In [31]:
query_4 = """
SELECT
    PICKUP_ZONE,
    ROUND(APPROX_PERCENTILE(AVG_TRIP_MINUTES, 0.5), 2) AS P50_DURATION,
    ROUND(APPROX_PERCENTILE(AVG_TRIP_MINUTES, 0.9), 2) AS P90_DURATION
FROM GOLD.FACT_TRIPS
GROUP BY PICKUP_ZONE
ORDER BY P90_DURATION DESC
LIMIT 10;
"""

df_4 = pd.read_sql(query_4, conn)
df_4


C:\Users\GISELLE1\AppData\Local\Temp\ipykernel_30484\515874390.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_4 = pd.read_sql(query_4, conn)


,PICKUP_ZONE,P50_DURATION,P90_DURATION
0,Westerleigh,114.00,114.00
1,Crotona Park,71.00,71.00
2,Hammels/Arverne,70.15,70.15
3,Far Rockaway,69.42,69.42
4,Woodlawn/Wakefield,59.88,59.88
5,Broad Channel,59.50,59.50
6,Country Club,59.00,59.00
7,Rockaway Park,58.79,58.79
8,Eastchester,57.65,57.65
9,Coney Island,57.21,57.21


El percentil 50 (mediana) varía entre 57 y 114 minutos, con el máximo en Westerleigh (114 min).

Zonas como Coney Island, Eastchester y Rockaway Park presentan viajes de duración media (57–59 min).

Los valores altos en zonas periféricas se deben a mayores distancias de traslado hacia el centro o tráfico costero.

La similitud entre p50 y p90 en algunas zonas sugiere baja dispersión, es decir, los viajes son consistentes en duración.

Elasticidad temporal

In [33]:
query_5 = """
SELECT
    TRIP_YEAR,
    TRIP_MONTH,
    PICKUP_BOROUGH,
    SUM(TOTAL_TRIPS) AS TOTAL_TRIPS
FROM GOLD.FACT_TRIPS
GROUP BY TRIP_YEAR, TRIP_MONTH, PICKUP_BOROUGH
ORDER BY TOTAL_TRIPS DESC
LIMIT 20;
"""

df_5 = pd.read_sql(query_5, conn)
df_5



C:\Users\GISELLE1\AppData\Local\Temp\ipykernel_30484\2530280278.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5 = pd.read_sql(query_5, conn)


,TRIP_YEAR,TRIP_MONTH,PICKUP_BOROUGH,TOTAL_TRIPS
0,2024,4,Manhattan,56490063
1,2024,4,Queens,6293700
2,2024,4,Brooklyn,638631
3,2024,4,None,209118
4,2024,4,Bronx,135870
5,2024,4,EWR,2016
6,2024,4,Staten Island,924
7,2024,3,Manhattan,105
8,2024,3,Queens,63
9,2002,12,Manhattan,21


En abril de 2024, Manhattan tuvo más de 56 millones de viajes, seguida de Queens (6.2 M) y Brooklyn (0.6 M).

Los meses de marzo y abril concentran la mayor parte del tráfico (pico de demanda).

Los registros de 2002 son residuales (21 viajes), sin relevancia estadística.

La caída progresiva posterior puede asociarse con limitaciones de muestreo o períodos no incluidos en la carga RAW.

In [35]:
conn.close()
print("Conexión cerrada. Análisis completado.")


Conexión cerrada. Análisis completado.
